In [32]:
import logging;

logging.basicConfig(level=logging.WARNING)
logging.getLogger().setLevel(level=logging.WARNING)

import warnings
warnings.simplefilter("ignore")

import gc
gc.enable()

In [33]:
from greenguard import get_pipelines

get_pipelines()

['resample_3600s_unstack_double_24_lstm_timeseries_classifier',
 'resample_3600s_unstack_24_lstm_timeseries_classifier',
 'resample_600s_unstack_144_lstm_timeseries_classifier',
 'resample_600s_normalize_dfs_1d_xgb_classifier',
 'resample_600s_unstack_dfs_1d_xgb_classifier',
 'resample_600s_unstack_double_144_lstm_timeseries_classifier',
 'resample_600s_unstack_normalize_dfs_1d_xgb_classifier']

In [41]:
from greenguard.demo import load_demo
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from greenguard.pipeline import GreenGuardPipeline


target_times, readings = load_demo()

template = 'resample_600s_unstack_double_144_lstm_timeseries_classifier'
window_size = 84
rule = '4h'

init_params = [{
    'pandas.DataFrame.resample#1': {
        'rule': rule,
    },
    'mlprimitives.custom.timeseries_preprocessing.cutoff_window_sequences#1': {
        'window_size': window_size,
    }
}]

In [42]:
def score_template(template, metric, target_times, readings, tuning_iterations,
                   init_params=None, cost=False, test_size=0.25, cv_splits=3, random_state=0):
    
    scores = {}
    
    try:
    
        train, test = train_test_split(target_times, test_size=test_size, random_state=random_state)

        pipeline = GreenGuardPipeline(template, metric, cost=cost, cv_splits=cv_splits, init_params=init_params)

        #Computing the default test score
        pipeline.fit(train, readings)
        predictions = pipeline.predict(test, readings)

        scores['default_test'] = f1_score(test['target'], predictions)

        #Computing de default cross validation score
        gc.collect()
        session = pipeline.tune(train, readings)
        session.run(1)

        scores['default_cv'] = pipeline.cv_score

        #Computing the cross validation score with tuned hyperparameters
        session.run(tuning_iterations)    
        pipeline.get_hyperparameters()

        scores['tuned_cv'] = pipeline.cv_score

        #Computing the test score with tuned hyperparameters
        pipeline.fit(train, readings)
        predictions = pipeline.predict(test, readings)

        scores['tuned_test']= f1_score(test['target'], predictions)
    
    except:
        return scores
    else:
        return scores

In [43]:
scores = score_template(
    template=template,
    metric=f1_score, 
    target_times=target_times, 
    readings=readings, 
    tuning_iterations=50,    
    init_params=init_params, 
    cost=False, 
    test_size=0.25, 
    cv_splits=3,
    random_state=0)

ERROR:mlblocks.mlpipeline:Exception caught fitting MLBlock keras.Sequential.DoubleLSTMTimeSeriesClassifier#1
Traceback (most recent call last):
  File "/home/usuario/.virtualenvs/GreenGuard/lib/python3.6/site-packages/mlblocks/mlpipeline.py", line 549, in _fit_block
    block.fit(**fit_args)
  File "/home/usuario/.virtualenvs/GreenGuard/lib/python3.6/site-packages/mlblocks/mlblock.py", line 302, in fit
    getattr(self.instance, self.fit_method)(**fit_kwargs)
  File "/home/usuario/.virtualenvs/GreenGuard/lib/python3.6/site-packages/mlprimitives/adapters/keras.py", line 111, in fit
    shuffle=self.shuffle)
  File "/home/usuario/.virtualenvs/GreenGuard/lib/python3.6/site-packages/keras/engine/training.py", line 1154, in fit
    batch_size=batch_size)
  File "/home/usuario/.virtualenvs/GreenGuard/lib/python3.6/site-packages/keras/engine/training.py", line 579, in _standardize_user_data
    exception_prefix='input')
  File "/home/usuario/.virtualenvs/GreenGuard/lib/python3.6/site-packages

In [37]:
#Result
scores

{}